爬蟲 需要的套件

In [18]:
#!pip install bs4
#!pip install requests
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import json
import time
import re
from urllib import request
import random

Redis 需要的套件

In [19]:
#!pip install redis
import redis
from hashlib import md5

In [20]:
class SimpleHash(object):
    def __init__(self, cap, seed):
        self.cap = cap
        self.seed = seed

    def hash(self, value):
        ret = 0
        for i in range(len(value)):
            ret += self.seed * ret + ord(value[i])
        return (self.cap - 1) & ret


class BloomFilter(object):
    def __init__(self, host='redis', port=6379, db=0, blockNum=1, key='bloomfilter'):
        """
        :param host: the host of Redis
        :param port: the port of Redis
        :param db: witch db in Redis
        :param blockNum: one blockNum for about 90,000,000; if you have more strings for filtering, increase it.
        :param key: the key's name in Redis
        """
        self.server = redis.Redis(host=host, port=port, db=db)
        self.bit_size = 1 << 31  # Redis的String类型最大容量为512M，现使用256M
        self.seeds = [5, 7, 11, 13, 31, 37, 61]
        self.key = key
        self.blockNum = blockNum
        self.hashfunc = []
        for seed in self.seeds:
            self.hashfunc.append(SimpleHash(self.bit_size, seed))

    def isContains(self, str_input):
        if not str_input:
            return False
        m5 = md5()
        m5.update(str_input)
        str_input = m5.hexdigest()
        ret = True
        name = self.key + str(int(str_input[0:2], 16) % self.blockNum)
        for f in self.hashfunc:
            loc = f.hash(str_input)
            ret = ret & self.server.getbit(name, loc)
        return ret

    def insert(self, str_input):
        m5 = md5()
        m5.update(str_input)
        str_input = m5.hexdigest()
        name = self.key + str(int(str_input[0:2], 16) % self.blockNum)
        for f in self.hashfunc:
            loc = f.hash(str_input)
            self.server.setbit(name, loc, 1)

BloomFilter方法:處理資料去重複

In [21]:
bf = BloomFilter()
# url = "https://tw.news.appledaily.com/politics/realtime/20181125/1473059/"
# url = url.encode('utf-8')
# if bf.isContains(url):   # 判断字符串是否存在
#     print('url is exists!')
# else:
#     print('url is not exists!')
#     bf.insert(url)

In [22]:
#BloomFilter的key
bf.key

'bloomfilter'

In [23]:
pool = redis.ConnectionPool(host='redis', port=6379, db=0)
r = redis.StrictRedis(connection_pool=pool)

#搜尋redis裡面有哪些keys
keys = r.keys()
print(keys)

#刪除redis裡key為XXX
r.delete("bloomfilter0")

傳送log 需要的套件

In [24]:
#!pip install python-logstash
import logging
import logstash
import sys

In [25]:
host = "logstash"
crawler_logger = logging.getLogger('crawler_logger')
crawler_logger.setLevel(logging.INFO)

# TCP
crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))

Kafka Producer 需要的套件

In [26]:
#!pip install kafka
from kafka import KafkaProducer
import sys
from kafka.errors import KafkaError

連線Kafka

In [27]:
producer = KafkaProducer(
    # Kafka集群在那裡?
    bootstrap_servers = ["kafka1:29092"],
    # 指定msgKey的序列化器, 若Key為None, 無法序列化, 透過producer直接給值
    key_serializer = str.encode,
    # 指定msgValue的序列化器
    value_serializer = lambda m: json.dumps(m).encode('utf-8')
)

TVBS 爬蟲方法

In [28]:
def tvbs_crawler2kafka():
    try:
        exit_limit = 0
        ###TVBS爬蟲開始###
        url = "https://news.tvbs.com.tw/politics/"
        response = urlopen(url)
        soup = BeautifulSoup(response)
        news_ul = soup.find_all("ul", id = "block_pc")[0]
        news_a = news_ul.find_all("a")

        news_no_string_list = []
        for a in news_a:
            news_no_string = a["data-news-id"]
            news_no_string_list.append(news_no_string)

        news_no_list = []
        for n in news_no_string_list:
            news_no_int = n.strip('\'')
            news_no_list.append(news_no_int)

        for news_no in news_no_list:

         #   print("新聞來源:", "TVBS新聞台")

            news_url = "https://news.tvbs.com.tw/politics/" + news_no
            print("新聞網址:", news_url)
            if exit_limit > 5:
                break

            #判斷網址是否爬過
            bf = BloomFilter()
            # Unicode-objects must be encoded before hashing
            news_url_u = news_url.encode('utf-8')
            if bf.isContains(news_url_u):   # 判断字符串是否存在
                print('url is exists!')
                exit_limit = exit_limit +1
                continue
            else:
                print('url is not exists!')
                bf.insert(news_url_u)


            req = requests.get(news_url)
            # requests如果找不到指定編碼，會猜測網頁編碼，有時會形成亂碼，故給指定編碼utf-8
            req.encoding = ("utf-8")
            soup = BeautifulSoup (req.text, 'html.parser')

            title = soup.find("h1", class_ = "margin_b20").text
            title = re.sub(r"\u3000", " ", title)
            title = re.sub(r"\xa0", " ", title)
         #   print("新聞標題:", title)

            date = soup.find("div", class_ = "icon_time time leftBox2").text
          #  print("發佈時間:", date)

            author = []
            author_text = soup.find("h4", class_ = "font_color5 leftBox1").text
            author_text = re.sub(r"攝影.*報導", "", author_text)
            author.append(author_text)
         #   print("作者:", author)

            content = soup.find("div", class_ = "h7 margin_b20").text
            content = content.replace("\t", "").replace("   ","").replace("\n", "").replace("（中央社）","")
            content = content.replace("最HOT話題在這！想跟上時事，快點我加入TVBS新聞LINE好友！ ","")
            content = content.replace("最HOT話題在這！想跟上時事，","").replace("快點我加入TVBS新聞LINE好友！ ","")
            content = content.replace("TVBS最新大數據分析","").replace("看完整內容快點我加入TVBS新聞LINE好友！","")
          #  print("內文:", content)

            keyword = soup.find("div", class_ = "adWords").text
            keyword = keyword.replace("\t", "")
            keyword = keyword.replace(",","")
            keyword = keyword.replace("\n", "").replace("編輯  ","").replace(" 報導","")
            keyword = keyword.replace("記者","").replace("      ",",")
            keyword = keyword.split(",")
        #    print("關鍵字", keyword)
            
            
            img = soup.find("div", class_ = "margin_b20")
            if not img == None:
                img_url = img.find('img')['src']
            else:
                img_url = ""
        #    print("圖片網址:", img_url)

            j = {"source": "TVBS新聞台" ,"url": news_url, "title": title, "date_": date, "author": author, "content": content, 
                 "kw": keyword, "img_url": img_url}
            time.sleep(1)

            # TCP連線到logstash
            crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
            # 爬蟲成功傳送至Elasticsearch的log訊息
            crawler_logger.info('python-crawler-logstash: TVBS crawler success!')
            crawler_logger.handlers.clear()
        ###TVBS爬蟲結束###
    
            # 步驟2.指定想要發佈訊息的topic名稱
            topic_name = "politics_news"

            print("Now sending messages ...")
            # 步驟3.產生要發佈到Kafka的訊息
            # - 參數  # 1: topicName
            # - 參數  # 2: msgKey
            # - 參數  # 3: msgValue

            producer.send(topic = topic_name, key = "tvbs", value = j)
            print("Message has finished sending!", "\n")
    except Exception as e:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
        # TCP連線到logstash
        crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
        # 爬蟲失敗傳送至Elasticsearch的log訊息
        crawler_logger.error('python-crawler-logstash: TVBS crawler error message!! [' + news_url + ']')
        crawler_logger.handlers.clear()

SETN爬蟲方法

In [29]:
def setn_crawler2kafka():
    try:
        exit_limit = 0
        ###SETN爬蟲開始###
        for page in range(1, 5):
            page_url = "https://www.setn.com/ViewAll.aspx?PageGroupID=6&p=" + str (page)
            print ("Processing:", page_url)

            response = urlopen (page_url)
            bs = BeautifulSoup (response)
            titles = bs.find_all ("h3", "view-li-title")
            if exit_limit > 5:
                break

            if len (titles) == 0:
                break

            for t in titles:

             #   print("新聞來源:", "SETN三立新聞網")

                news_url = "https://www.setn.com" + t.find ("a")["href"]
                print("新聞網址:", news_url)


                #判斷網址是否爬過
                bf = BloomFilter()
                # Unicode-objects must be encoded before hashing
                news_url_u = news_url.encode('utf-8')
                if bf.isContains(news_url_u):   # 判断字符串是否存在
                    print('url is exists!')
                    exit_limit = exit_limit +1
                    continue
                else:
                    print('url is not exists!')
                    bf.insert(news_url_u)


                req = requests.get (news_url)
                bs = BeautifulSoup (req.text, 'html.parser')

                title = bs.find('h1', class_ = "news-title-3").text
                title = re.sub(r"\u3000", " ", title)
           #     print("新聞標題:", title)

                date = bs.find("time", class_ = "page-date").text.strip()
          #      print("發佈時間:", date)

                author = []
                a_divs = bs.find("div", id="Content1")
                first_p = a_divs.find("p")
                author_text = first_p.text
                #找第一段有沒有含"／"，如果沒有給予空字串
                if author_text.find("／") == -1:
                    author_text = ""
                #如果作者是空字串，給予空list
                if author_text == "":
                    author = []
                else:
                    author.append(author_text)
               # print("作者:", author)

                c_divs = bs.find('div', itemprop='articleBody')
                content_all = c_divs.find_all ("p")
                content = ""
                for c in content_all:
                    if c.attrs == {}:
                        content = content + c.text
                        #print(c.text)

                        c_divs = bs.find('div', itemprop='articleBody')
                content_all = c_divs.find_all ("p")
                content = ""
                for c in content_all:
                    if c.attrs == {}:
                        content = content + c.text
                        #print(c.text)
             #   print("內文:", content)

                keyword = []
                k_divs = bs.find("div", class_="keyword page-keyword-area")
                k_strong = k_divs.find_all("strong")
                for k in k_strong:
                    keyword_text = k.text
                    keyword.append(keyword_text)
               # print("關鍵字:", keyword)
                
                
                img_p = bs.find("p", style = "text-align: center;")
                if not img_p == None:
                    img_url = img_p.find("img")["src"]
                else:
                    img_url = ""
              #  print("圖片網址:", img_url)

                j = {"source": "SETN三立新聞網" ,"url": news_url, "title": title, "date_": date, "author": author, "content": content, 
                     "kw": keyword, "img_url": img_url}
                time.sleep (1)

                # TCP連線到logstash
                crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
                # 爬蟲成功傳送至Elasticsearch的log訊息
                crawler_logger.info('python-crawler-logstash: SETN crawler success!')
                crawler_logger.handlers.clear()
        ###SETN爬蟲結束###
    
                # 步驟2.指定想要發佈訊息的topic名稱
                topic_name = "politics_news"

                print("Now sending messages ...")
                # 步驟3.產生要發佈到Kafka的訊息
                # - 參數  # 1: topicName
                # - 參數  # 2: msgKey
                # - 參數  # 3: msgValue

                producer.send(topic = topic_name, key = "setn", value = j)
                print("Message has finished sending!", "\n")
    except Exception as e:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
        # TCP連線到logstash
        crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
        # 爬蟲失敗傳送至Elasticsearch的log訊息
        crawler_logger.error('python-crawler-logstash: SETN crawler error message!! [' + news_url + ']')
        crawler_logger.handlers.clear()

AppleDaily爬蟲方法

In [30]:
def appledaily_crawler2kafka():
    try:
        exit_limit = 0
        ###AppleDaily爬蟲開始###
        for page in range(1, 12):
            time.sleep(2)
            href = "https://tw.news.appledaily.com/politics/realtime/" + str(page)
            res = requests.get(href)
            html = BeautifulSoup(res.text)
            if exit_limit > 5:
                break

            all_news_1 =  html.find_all("ul", class_= "rtddd slvl")
            for all_news in all_news_1:
                news = all_news.find_all("a")
                #print("第", page, "頁")
                for n in news:

                  #  print("新聞來源:", "蘋果日報")

                    #my_news = {}
                    news_url ="https://tw.news.appledaily.com/" + str(n["href"])
                    print("新聞網址:", news_url)

                    #判斷網址是否爬過
                    bf = BloomFilter()
                    # Unicode-objects must be encoded before hashing
                    news_url_u = news_url.encode('utf-8')
                    if bf.isContains(news_url_u):   # 判断字符串是否存在
                        print('url is exists!')
                        exit_limit = exit_limit +1
                        continue
                    else:
                        print('url is not exists!')
                        bf.insert(news_url_u)

                    news_per = requests.get(news_url)
                    bs = BeautifulSoup(news_per.text)

                    title = bs.find("h1").text
                    title = re.sub(r"\u3000", " ", title)
                    #print("新聞標題:", title)           

                    date = bs.find("div", class_="ndArticle_creat").text.replace("出版時間：", "")
                    #print("發佈時間:", date)

                    if not bs.find("div", class_="ndgKeyword") == None:
                        key_word = bs.find("div", class_="ndgKeyword").find_all("a")
                        key_list = []
                        for k in key_word:
                            key_list.append(k.text)
                    else:
                        key_list = []       

                    content_dir = bs.find("div", class_="ndArticle_margin")
                    for a in content_dir.find_all("a"):
                        if not a == None:
                            a.extract()
                    for s in content_dir.find_all("span"):
                        if not s == None:
                            s.extract()
                    for i in content_dir.find_all("iframe"):
                        if not i == None:
                            i.extract()
                    for d in content_dir.find_all("div"):
                        if not d == None:
                            d.extract()
                    for st in content_dir.find_all("style"):
                        if not st == None:
                            st.extract()
                    content = content_dir.text.strip()
                    content = content.replace("）", ")").replace("（", "(").replace("／", "/").replace("╱", "/")       
                    pat = re.compile(r"\(((.{0,15})/.{0,8}[\u5831][\u5c0e])\)")
                    pat_2 = re.compile(r"【((.{0,15})/.{0,8}[\u5831][\u5c0e])】")
                    ans_news = pat.search(content)
                    ans_news_2 = pat_2.search(content)
                    if ans_news == None:
                        author = []
                    elif not ans_news_2 == None:
                        author = ans_news[2].split("、")
                    else:
                        #print(ans_news[2])
                        author = ans_news[2].split("、")
                        content = content.split(ans_news[0])[0]
                    #print("作者:", author)   
                    #print("內文:", content)  
                   # print("關鍵字:", key_list)

                    img_box = bs.find("div", class_="ndAritcle_headPic")
                    if not img_box == None:
                        img_url = img_box.find("img")['src']
                    else:
                        img_url = ""
                    #print("圖片網址:", img_url)

                    j = {"source": "蘋果日報" ,"url": news_url, "title": title, "date_": date, "author": author, 
                         "content": content, "kw": key_list, "img_url": img_url}
                    time.sleep(random.randint(1, 2))

                    # TCP連線到logstash
                    crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
                    # 爬蟲成功傳送至Elasticsearch的log訊息
                    crawler_logger.info('python-crawler-logstash: AppleDaily crawler success!')
                    crawler_logger.handlers.clear()
        ###AppleDaily爬蟲結束###

                    # 步驟2.指定想要發佈訊息的topic名稱
                    topic_name = "politics_news"

                    print("Now sending messages ...")
                    # 步驟3.產生要發佈到Kafka的訊息
                    # - 參數  # 1: topicName
                    # - 參數  # 2: msgKey
                    # - 參數  # 3: msgValue

                    producer.send(topic = topic_name, key = "appledaily", value = j)
                    print("Message has finished sending!", "\n")
    except Exception as e:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
        # TCP連線到logstash
        crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
        # 爬蟲失敗傳送至Elasticsearch的log訊息
        crawler_logger.error('python-crawler-logstash: AppleDaily crawler error message!! [' + news_url + ']')
        crawler_logger.handlers.clear()

NOWnews爬蟲方法

In [31]:
def nownews_crawler2kafka():
    try:
        exit_limit = 0
        ###NOWnews爬蟲開始###
        for page in range(1,6):
            href = 'https://www.nownews.com/cat/politics/page/'+str(page)

            response = requests.get(href)
            soup=BeautifulSoup(response.text)
            news = soup.find_all("h3",class_="entry-title td-module-title")
            if exit_limit > 5:
                break

            for n in news:
                
                #print("新聞來源:", "NOWnews")

                news_url = n.find("a")["href"]
                print("新聞網址:", news_url)


                #判斷網址是否爬過
                bf = BloomFilter()
                # Unicode-objects must be encoded before hashing
                news_url_u = news_url.encode('utf-8')
                if bf.isContains(news_url_u):   # 判断字符串是否存在
                    print('url is exists!')
                    exit_limit = exit_limit +1
                    continue
                else:
                    print('url is not exists!')
                    bf.insert(news_url_u)


                response = requests.get(news_url)
                html = BeautifulSoup(response.text)
                title = html.find("h1",class_="entry-title").text
                title = re.sub("\u3000", " ", title)
                #print("新聞標題:", title)

                date = html.find("time",class_ = "entry-date").text
                #print("發佈時間:", date)

                author = []
                author_text = html.find("div",class_="td-post-author-name").text
                author_text = author_text.split("／")[0]
                author_text = author_text.split("/")[0].replace("記者","").replace("\n","").replace(" ","")
                author.append(author_text)
                #print("作者:", author)

                te = html.find("div", class_="td-post-content").contents
                #print(te)
                #print(te[0].name == None)
                content = ""
                for a in te:
                    if a.name == "p":
                        content = content + a.text
                #print("內文:", content)

                #處理keyword
                if not html.find("ul",class_="td-tags") == None:
                    kw = html.find("ul",class_="td-tags").find_all("a")
                    keyword = []
                    for k in kw:
                        keyword.append(k.text)
                else:
                    keyword = []
                #print("關鍵字:", keyword)

                img_url = html.find("div",class_="td-post-featured-image")
                if not img_url == None:
                    img_url = img_url.find("img")["src"]
                else:
                    img_url = ""
                #print("圖片網址:", img_url)

                j = {"source": "NOWnews" ,"url": news_url, "title": title, "date_": date, "author": author, "content": content, 
                     "kw": keyword, "img_url": img_url}
                time.sleep(random.randint(0,2))

                # TCP連線到logstash
                crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
                # 爬蟲成功傳送至Elasticsearch的log訊息
                crawler_logger.info('python-crawler-logstash: NOWnews crawler success!')
                crawler_logger.handlers.clear()
        ###NOWnews爬蟲結束###
    
                # 步驟2.指定想要發佈訊息的topic名稱
                topic_name = "politics_news"

                print("Now sending messages ...")
                # 步驟3.產生要發佈到Kafka的訊息
                # - 參數  # 1: topicName
                # - 參數  # 2: msgKey
                # - 參數  # 3: msgValue

                producer.send(topic = topic_name, key = "nownews", value = j)
                print("Message has finished sending!", "\n")
    except Exception as e:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
        # TCP連線到logstash
        crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
        # 爬蟲失敗傳送至Elasticsearch的log訊息
        crawler_logger.error('python-crawler-logstash: NOWnews crawler error message!! [' + news_url + ']')
        crawler_logger.handlers.clear()

STORM爬蟲方法

In [32]:
def storm_crawler2kafka():
    try:
        exit_limit = 0
        ###STORM爬蟲開始###    
        for page in range(1,5):
            href = 'https://www.storm.mg/category/118/'+str(page)

            response = urlopen(href)
            html = BeautifulSoup(response)
            #找到特定範圍在選取網址列
            newf = html.find("div",class_ = "middle_category_cards")
            #選取所有單篇網址列
            news = newf.find_all("div",class_ = "category_card")
            #迴圈抓取整頁新聞連結
            if exit_limit > 5:
                break
            for r in news:

                #print("新聞來源:", "風傳媒 THE STORM MEDIA")

                news_url = r.find("a",class_ = "card_link")["href"]
                print("新聞網址:", news_url)


                #判斷網址是否爬過
                bf = BloomFilter()
                # Unicode-objects must be encoded before hashing
                news_url_u = news_url.encode('utf-8')
                if bf.isContains(news_url_u):   # 判断字符串是否存在
                    print('url is exists!')
                    exit_limit = exit_limit +1
                    continue
                else:
                    print('url is not exists!')
                    bf.insert(news_url_u)


                response = urlopen(news_url)
                html = BeautifulSoup(response)

                title = html.find("h1", id = "article_title").text
                #print("新聞標題:", title)

                date = html.find("span", class_="info_time").text
                #print("發佈時間:", date)

                author = []
                author_text = html.find("span", class_="info_author").text.replace("新新聞","")
                author.append(author_text)
                #print("作者:", author)

                content = html.find("div", id="CMS_wrapper").text.replace("\n","").replace("\t","")
                #print("內文:", content)

                keyword = html.find("div", id="tags_list_wrapepr").text.split()
                #print("關鍵字:", keyword)

                img_url = html.find("img", id="feature_img")
                if not img_url == None:
                    img_url = img_url["src"]
                else:
                    img_url = ""
               # print("圖片網址:", img_url)
                
                j = {"source": "風傳媒 THE STORM MEDIA" ,"url": news_url, "title": title, "date_": date, "author": author, 
                     "content": content, "kw": keyword, "img_url": img_url}
                time.sleep(random.randint(0,2))

                # TCP連線到logstash
                crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
                # 爬蟲成功傳送至Elasticsearch的log訊息
                crawler_logger.info('python-crawler-logstash: STORM crawler success!')
                crawler_logger.handlers.clear()
        ###STORM爬蟲結束###
    
                # 步驟2.指定想要發佈訊息的topic名稱
                topic_name = "politics_news"

                print("Now sending messages ...")
                # 步驟3.產生要發佈到Kafka的訊息
                # - 參數  # 1: topicName
                # - 參數  # 2: msgKey
                # - 參數  # 3: msgValue

                producer.send(topic = topic_name, key = "storm", value = j)
                print("Message has finished sending!", "\n")
            
    except Exception as e:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
        # TCP連線到logstash
        crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
        # 爬蟲失敗傳送至Elasticsearch的log訊息
        crawler_logger.error('python-crawler-logstash: STORM crawler error message!! [' + news_url + ']')
        crawler_logger.handlers.clear()

ETtoday爬蟲方法

In [34]:
def ettoday_crawler2kafka():
    try:
        exit_limit = 0
        ###ETtoday爬蟲開始###
        for page in range(1, 250):
            url = "https://www.ettoday.net/news_search/doSearch.php?keywords=%E6%94%BF%E6%B2%BB&kind=1" + "&idx=1&page=" + str(
                page)
            #print("第", page, "頁"," ","Processing: ", url)
            try:
                response = urlopen(url)
            except HTTPError:
                #print("大概是結束了")
                break
            html = BeautifulSoup(response)

            all_ar = html.find("div", class_="result_archive")

            val = all_ar.find_all("div", class_="archive clearfix")
            if exit_limit > 5:
                break

            for u in val:

               # print("新聞來源:", "ETtoday")

                box2 = u.find("div", class_="box_2")
                news_url = box2.find("a")["href"]
                print("新聞網址:", news_url)


                #判斷網址是否爬過
                bf = BloomFilter()
                # Unicode-objects must be encoded before hashing
                news_url_u = news_url.encode('utf-8')
                if bf.isContains(news_url_u):   # 判断字符串是否存在
                    print('url is exists!')
                    exit_limit = exit_limit +1
                    continue
                else:
                    print('url is not exists!')
                    bf.insert(news_url_u)


                a_response = urlopen(news_url)
                a_html = BeautifulSoup(a_response, "html.parser")

                title = a_html.find("h1", class_="title").text
                title = re.sub("\u3000", " ", title)
                #print("新聞標題:", title)

                date = a_html.find("time", class_="date").text.strip()
                date = re.sub("年", "/", date)
                date = re.sub("月", "/", date)
                date = re.sub("日", "", date)
                #print("發佈時間:", date)


                a_story = a_html.find("div", class_="story")                
                content_p = a_story.find_all("p")
                author_c = ""
                for c_p in content_p:
                    #如果c_p裡面的第一個格子是None
                    if c_p.contents[0].name == None:
                        author_c = author_c + c_p.text
                    elif c_p.contents[0].name == "span":
                        author_c = author_c + c_p.text
                author = []
                if re.search(r".*／.?.?報導", author_c) == None:
                    author = []
                else:
                    #因為作者與內文會放在同一個格子，所以將內文中會抓到的作者部分在內文處理時刪除
                    content_del = re.search(r".*／.?.?報導", author_c).group()
                    author_text = re.search(r".*／.?.?報導", author_c).group() 
                author.append(author_text)
                #print("作者:", author)              

                content = ""
                for c in content_p:
                    if c.contents[0].name == None:
                        content = content + c.text
                        content = content.replace("\u3000", " ")      
                        content = content.replace(content_del, "")            
                #print("內文:", content)

                keyword = []
                a_web = a_html.find("article")
                a_kw = a_web.find("p", class_="tag")
                kw = a_kw.find_all("a", target="_blank")
                for k in kw:
                    kw_n = k.text
                    keyword.append(kw_n)
                if keyword == [""]:
                    keyword = []
                #print("關鍵字:", keyword)

                if not a_story.find("img")['src'] == None:
                    img_url = a_story.find("img")['src']
                    img_url = "https:" + img_url
                else:
                    img_url = ""
                #print("圖片網址:", img_url)

                j = {"source": "ETtoday" ,"url": news_url, "title": title, "date_": date, "author": author, "content": content, 
                     "kw": keyword, "img_url": img_url}
                time.sleep(1)

                # TCP連線到logstash
                crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
                # 爬蟲成功傳送至Elasticsearch的log訊息
                crawler_logger.info('python-crawler-logstash: ETtoday crawler success!')
                crawler_logger.handlers.clear()
        ###ETtoday爬蟲結束###

                # 步驟2.指定想要發佈訊息的topic名稱
                topic_name = "politics_news"

                print("Now sending messages ...")
                # 步驟3.產生要發佈到Kafka的訊息
                # - 參數  # 1: topicName
                # - 參數  # 2: msgKey
                # - 參數  # 3: msgValue

                producer.send(topic = topic_name, key = "ettoday", value = j)
                print("Message has finished sending!", "\n")
    except Exception as e:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
        # TCP連線到logstash
        crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
        # 爬蟲失敗傳送至Elasticsearch的log訊息
        crawler_logger.error('python-crawler-logstash: ETtoday crawler error message!! [' + news_url + ']')
        crawler_logger.handlers.clear()

所有爬蟲執行方法

In [35]:
def all_crawler():
    tvbs_crawler2kafka()
    setn_crawler2kafka()
    appledaily_crawler2kafka()
    nownews_crawler2kafka()
    storm_crawler2kafka()
    ettoday_crawler2kafka()

In [ ]:
ettoday_crawler2kafka()

主程式

In [ ]:
#!pip install schedule
import schedule
import time
import datetime
import threading

schedule.every(10).minutes.do(all_crawler)
#def job():
#    print("I'm working...")
#schedule.every(1).minutes.do(job)
#schedule.every().day.at("16:42").do(tvbs_crawler2kafka)
#schedule.every().day.at("17:51").do(setn_crawler2kafka)
#schedule.every().day.at("22:27").do(appledaily_crawler2kafka) 
#schedule.every().day.at("22:27").do(nownews_crawler2kafka)
#schedule.every().day.at("22:27").do(storm_crawler2kafka())

while True:
    schedule.run_pending()
    time.sleep(1)


新聞網址: https://news.tvbs.com.tw/politics/1057965
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://news.tvbs.com.tw/politics/1058009
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://news.tvbs.com.tw/politics/1057932
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://news.tvbs.com.tw/politics/1057945
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://news.tvbs.com.tw/politics/1057946
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://news.tvbs.com.tw/politics/1057952
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://news.tvbs.com.tw/politics/1057947
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://news.tvbs.com.tw/politics/1057960
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: ht

Now sending messages ...
Message has finished sending! 

新聞網址: https://news.tvbs.com.tw/politics/1057281
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://news.tvbs.com.tw/politics/1057305
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://news.tvbs.com.tw/politics/1057295
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://news.tvbs.com.tw/politics/1057289
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://news.tvbs.com.tw/politics/1057242
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://news.tvbs.com.tw/politics/1057236
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://news.tvbs.com.tw/politics/1057196
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://news.tvbs.com.tw/politics/1057166
url is not exists!
Now send

Now sending messages ...
Message has finished sending! 

Processing: https://www.setn.com/ViewAll.aspx?PageGroupID=6&p=2
新聞網址: https://www.setn.com/News.aspx?NewsID=478569
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.setn.com/News.aspx?NewsID=478570
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.setn.com/News.aspx?NewsID=478560
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.setn.com/News.aspx?NewsID=478568
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.setn.com/News.aspx?NewsID=478566
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.setn.com/News.aspx?NewsID=478565
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.setn.com/News.aspx?NewsID=478555
url is not exists!
Now sending messages ...
Message has finished s

Now sending messages ...
Message has finished sending! 

新聞網址: https://www.setn.com/News.aspx?NewsID=478103
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.setn.com/News.aspx?NewsID=478075
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.setn.com/News.aspx?NewsID=478040
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.setn.com/News.aspx?NewsID=478063
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.setn.com/News.aspx?NewsID=478038
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.setn.com/News.aspx?NewsID=478082
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.setn.com/News.aspx?NewsID=478009
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.setn.com/News.aspx?NewsID=478081
url

Now sending messages ...
Message has finished sending! 

新聞網址: https://tw.news.appledaily.com//politics/realtime/20190101/1493137/
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://tw.news.appledaily.com//politics/realtime/20190101/1493037/
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://tw.news.appledaily.com//politics/realtime/20190101/1493065/
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://tw.news.appledaily.com//politics/realtime/20190101/1493066/
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://tw.news.appledaily.com//politics/realtime/20190101/1493090/
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://tw.news.appledaily.com//politics/realtime/20190101/1493097/
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://tw.news.appledaily.com//polit

...

Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181224/1338807.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181224/1338674.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181223/1338704.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181223/1338717.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181223/1338585.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181223/1338646.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181223/1338528.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.e

Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181221/1336791.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181221/1336831.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181221/1336522.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181221/1336645.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181221/1336650.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181221/1336524.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181221/1336711.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.e

Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181219/1334866.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181219/1334911.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181219/1334865.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181219/1334759.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181219/1334855.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181219/1334595.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181219/1334783.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.e

新聞網址: https://www.ettoday.net/news/20181218/1333534.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181218/1333560.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181218/1333533.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181218/1333537.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181218/1333505.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181218/1333460.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181217/1333456.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181217/1333417.htm
url is not exists!
N

Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181215/1331973.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181215/1331902.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181215/1331943.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181215/1331939.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181215/1331784.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181215/1331850.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.ettoday.net/news/20181215/1331709.htm
url is not exists!
Now sending messages ...
Message has finished sending! 

新聞網址: https://www.e

In [ ]:
# coding=utf-8
import threading, queue, time, urllib
from  urllib import request
baseUrl = 'http://www.pythontab.com/html/pythonjichu/'
urlQueue = queue.Queue()
for i in range(2, 10):
    url = baseUrl + str(i) + '.html'
    urlQueue.put(url)
    #print(url)
def fetchUrl(urlQueue):
    while True:
        try:
            #不阻塞的读取队列数据
            url = urlQueue.get_nowait()
            i = urlQueue.qsize()
        except Exception as e:
            break
        print ('Current Thread Name %s, Url: %s ' % (threading.currentThread().name, url))
        try:
            response = urllib.request.urlopen(url)
            responseCode = response.getcode()
        except Exception as e:
            continue
        if responseCode == 200:
            #抓取内容的数据处理可以放到这里
            #为了突出效果， 设置延时
            time.sleep(1)
if __name__ == '__main__':
    startTime = time.time()
    threads = []
    # 可以调节线程数， 进而控制抓取速度
    threadNum = 2
    for i in range(0, threadNum):
        t = threading.Thread(target=fetchUrl, args=(urlQueue,))
        threads.append(t)
    for t in threads:
        t.start()
    for t in threads:
        #多线程多join的情况下，依次执行各线程的join方法, 这样可以确保主线程最后退出， 且各个线程间没有阻塞
        t.join()
    endTime = time.time()
    print ('Done, Time cost: %s ' %  (endTime - startTime))